In [1]:
#cleaning data
import pandas as pd
import numpy as np
import datetime
import re
import os
import time
import boto3
#prep for model
from sklearn.feature_extraction.text import CountVectorizer
#pulling in the model
from sklearn.externals import joblib

# ETL

In [8]:
def clean_comments(text):
    '''
    This function will remove the common text before the comments
    '''
    pattern_1 = r"ON LtOut WE\d{4,5} SAYS\s"
    regex_1 = re.sub(pattern_1, '', text)
    pattern_2 = r"ON LtOut TRBL SAYS"
    regex_2 = re.sub(pattern_2, '', regex_1)
    pattern_3 = r"ON HAZ TRBL SAYS"
    regex_3 = re.sub(pattern_3, '', regex_2)
    new_text = regex_3.replace('@', 'at')
    pattern_4 = r"[^a-zA-Z0-9]+"
    return re.sub(pattern_4, ' ', new_text)

In [12]:
#def lambda_handler(event, context):
#    '''
#    This function will import and run data throught the nlp model
#    '''
    #set the working directory to where it can read and write
os.chdir('/tmp/')

#make an s3 object
s3 = boto3.client('s3')
#get the bucket from the event object
bucket = 'distribution-reliability-nlp' # event['Records'][0]['s3']['bucket']['name']

asset_files = s3.list_objects(Bucket=bucket, Prefix='testing/assets/')['Contents']

for file in asset_files:
    the_file = file['Key']
    file_name = the_file.split('/')[-1]
    s3.download_file(bucket, the_file, file_name)

In [13]:
#get to day's date
today = datetime.datetime.today()

#file names in s3
we_file = 'We-Outages_{}_{}_{}.xlsx'.format(today.year, 
                                            today.month,
                                           today.day)

wps_file = 'WPS-Outages_{}_{}_{}.xlsx'.format(today.year, 
                                            today.month,
                                           today.day)

#list of file names
all_files = [we_file, wps_file]


#copy model results from s3 to instance
for file in all_files:
    import_location = 'testing/import/{}'.format(file)
    s3.download_file(bucket, import_location, file)

In [14]:
#import data
we_outages = pd.read_excel(we_file)
wps_outages = pd.read_excel(wps_file)


#save raw data to new s3 location
we_response = s3.upload_file(we_file,
                                Bucket=bucket, 
                                Key='testing/raw/{}'.format(we_file))

wps_response = s3.upload_file(wps_file,
                                Bucket=bucket, 
                                Key='testing/raw/{}'.format(wps_file))
print(we_outages.shape)
print(wps_outages.shape)

(172, 48)
(132, 60)


In [21]:
#columns to keep in each file                               
we_save = we_outages[['outage_id', 'mobil_comments']]
wps_save = wps_outages[['Event', 'ClosureRemarks']]


we_save = we_save.rename(columns={'outage_id': 'Outage ID'})
wps_save = wps_save.rename(columns={'Event': 'Outage ID', 'ClosureRemarks':'mobil_comments'})


#adding company so we can work with one dataframe
we_save['Company'] = 'We Energies'
wps_save['Company'] = 'WPS'

print(we_save.shape)
print(wps_save.shape)

(172, 3)
(132, 3)


In [23]:
#make one dataframe
all_outages = pd.concat([we_save, wps_save], sort=True)

#make sure comments are strings
all_outages['mobil_comments'] = all_outages['mobil_comments'].map(str)
#clean comments
all_outages['mobil_comments'] = all_outages['mobil_comments'].apply(clean_comments)       

#drop missings
all_outages = all_outages.dropna()
all_outages = all_outages[all_outages['mobil_comments'] != 'nan']
all_outages = all_outages[all_outages['mobil_comments'] != ' ']
all_outages.head()

,Company,Outage ID,mobil_comments
0,We Energies,3302912,ON LtOut W39069 SAYS REPAIRED 3 0 URD FAULT CA...
1,We Energies,3302999,REPLACED 3 OF 3 AT BUILDING Repair OH CONNECT...
2,We Energies,3302997,ON LtOut W41570 SAYS REPLACE OH TUB FUSE SQUIR...
3,We Energies,3303010,ON LtOut W41570 SAYS REPLACED 40ARF AT P 61 57...
4,We Energies,3303023,ON LtOut W36062 SAYS UPON ARRIVAL THE CALL WAS...


In [25]:
#file name
model_file_location = ('Model_{}_{}_{}.csv').format(today.year, 
                                                         today.month,
                                                        today.day)

#save to csv
all_outages.to_csv(model_file_location, index=False)

#svae to s3
results_response = s3.upload_file(Filename=model_file_location,
                                Bucket=bucket, 
                                Key='testing/to_model/{}'.format(model_file_location))

#return print('Data is Ready for model.')

# Run Model

In [26]:
def category_name(data, decoder_obj):
    '''
    This function takes model predictions and returns the word category
    '''
    result = []
    for pred in data:
        result.append(decoder_obj[pred])
    return result 

In [94]:
def get_decoder(general_category):
    decoder_list = []
    f = open(general_category + '.txt', 'r')
    f1 = f.readlines()
    for x in f1:
        x = x[:-1]
        decoder_list.append(x)

    if general_category == 'General':
        decoder_list.remove('')

    decoder_dict = {}
    for i in range(0, len(decoder_list)):
        decoder_dict[i] = decoder_list[i]

    return decoder_dict

In [50]:
def predict_general_category(data):
    #import text vectorizer
    count_vectorizer = joblib.load('General-count-vectorizer.pkl')
    #import model
    model = joblib.load('General-model.pkl')  

    #vectorize text
    X = count_vectorizer.transform(data['mobil_comments'])

    # Use the loaded model to make predictions
    pred = model.predict(X)
    pred_prob = model.predict_proba(X)

    confidence = []
    for i in range(0,len(pred_prob)):
        confidence.append(pred_prob[i][pred[i]])

    #convert the numberic categories back to words
    decoder = get_decoder('General')

    #convert the numberic categories back to words
    data['General Predictions'] = category_name(pred, decoder)
    data['General Confidence'] = confidence

    return data

In [99]:
def predict_sub_category(data, category):
    
    #limit the data to input category
    data = data[data['General Predictions'] == str(category)]
    
    if category == 'Planned':
        data['Subcategory Predictions'] = 'New Construction'
        return data
    elif category == 'Power-Supply':
        data['Subcategory Predictions'] = 'Transmission'
        return data
    else:
        #import text vectorizer
        count_vectorizer = joblib.load(category + '-count-vectorizer.pkl')
        #import model
        model = joblib.load(category +'-model.pkl')  
        
        #vectorize text
        X = count_vectorizer.transform(data['mobil_comments'])

        # Use the loaded model to make predictions
        #try:
        pred = model.predict(X)
        pred_prob = model.predict_proba(X)

        confidence = []
        for i in range(0,len(pred_prob)):
            confidence.append(pred_prob[i][pred[i]])
        #except:
         #   data[var_name + ' Predictions'] = ''
         #   data[var_name + ' Confidence'] = ''
         #   return data
    
        #get number to word decoder
        decoder = get_decoder(category)

        #convert back to word
        data['Subcategory Predictions'] = category_name(pred, decoder)
        data['Subcategory Confidence'] = confidence

        return data

In [80]:
#set the working directory to where it can read and write

#make an s3 object
s3 = boto3.client('s3')
#get the bucket from the event object
bucket = 'distribution-reliability-nlp' #event['Records'][0]['s3']['bucket']['name']

asset_files = s3.list_objects(Bucket=bucket, Prefix='testing/assets/')['Contents']

for file in asset_files:
    the_file = file['Key']
    file_name = the_file.split('/')[-1]
    s3.download_file(bucket, the_file, file_name)


In [96]:
#get to day's date
today = datetime.datetime.today()

#file names in s3
model_file = 'Model_{}_{}_{}.csv'.format(today.year, 
                                        today.month,
                                         today.day)


import_location = 'testing/to_model/{}'.format(model_file)
s3.download_file(bucket, import_location, model_file)

#import data
all_outages = pd.read_csv(model_file)
print(all_outages.shape)
all_outages.head()

(300, 3)


,Company,Outage ID,mobil_comments
0,We Energies,3302912,ON LtOut W39069 SAYS REPAIRED 3 0 URD FAULT CA...
1,We Energies,3302999,REPLACED 3 OF 3 AT BUILDING Repair OH CONNECT...
2,We Energies,3302997,ON LtOut W41570 SAYS REPLACE OH TUB FUSE SQUIR...
3,We Energies,3303010,ON LtOut W41570 SAYS REPLACED 40ARF AT P 61 57...
4,We Energies,3303023,ON LtOut W36062 SAYS UPON ARRIVAL THE CALL WAS...


In [97]:
all_outages = predict_general_category(all_outages)
print(all_outages.shape)
all_outages.head()

(300, 5)


,Company,Outage ID,mobil_comments,General Predictions,General Confidence
0,We Energies,3302912,ON LtOut W39069 SAYS REPAIRED 3 0 URD FAULT CA...,Equipment,0.938445
1,We Energies,3302999,REPLACED 3 OF 3 AT BUILDING Repair OH CONNECT...,Equipment,0.819342
2,We Energies,3302997,ON LtOut W41570 SAYS REPLACE OH TUB FUSE SQUIR...,Wildlife,0.997352
3,We Energies,3303010,ON LtOut W41570 SAYS REPLACED 40ARF AT P 61 57...,Wildlife,0.997923
4,We Energies,3303023,ON LtOut W36062 SAYS UPON ARRIVAL THE CALL WAS...,Equipment,0.457667


In [100]:
#loop through general categories and predict the subcategories
results_list = []
for types in list(all_outages['General Predictions'].unique()):
    print(types)
    results_list.append(predict_sub_category(all_outages, types))
    
#combine all subcategory predictions
results_df = pd.concat(results_list, sort=True)

#limit the dataframe to what is needed
results_df = results_df[['Outage ID', 'Company', 
                        'General Predictions', 'General Confidence', 
                        'Subcategory Predictions', 'Subcategory Confidence']]
print(results_df.shape)
results_df.head()

Equipment
Wildlife


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Weather
Public
Vegetation
Planned
(300, 6)


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Outage ID,Company,General Predictions,General Confidence,Subcategory Predictions,Subcategory Confidence
0,3302912,We Energies,Equipment,0.938445,Conductor/Cable,0.982703
1,3302999,We Energies,Equipment,0.819342,Connector/Splice,0.867770
4,3303023,We Energies,Equipment,0.457667,Simple Interrupting Device,0.813357
7,3303302,We Energies,Equipment,0.887896,Conductor/Cable,0.933364
8,3303715,We Energies,Equipment,0.929131,Conductor/Cable,0.992272


In [101]:
#add columns for human review eval
results_df['Was the Outage Reportable?'] = ''
results_df['Reportable Reference Outage ID'] = ''
results_df['Did You Change the General Category?'] = ''
results_df['What General Category did you change it to?']
results_df['Was the General Prediction Correct?'] = ''
results_df['Did You Change the Subcategory Category?'] = ''
results_df['Was the Subcategory Prediction Correct?'] = ''    

#file name
results_file_location = ('NLP-Results_{}_{}_{}.csv').format(today.year, 
                                                         today.month,
                                                        today.day)

#save to csv
results_df.to_csv(results_file_location, index=False)

#svae to s3
results_response = s3.upload_file(Filename=results_file_location,
                                Bucket=bucket, 
                                Key='testing/export/{}'.format(results_file_location))

#return print('Predictions Made.')